In [1]:
from hydradx.model.amm.omnipool_amm import OmnipoolState
from hydradx.model.amm.agents import Agent
from hydradx.model.processing import parse_kraken_orderbook
from hydradx.model.amm.arbitrage_agent import get_arb_swaps, calculate_profit, execute_arb
from hydradx.model.amm.centralized_market import CentralizedMarket, OrderBook
from hydradxapi import HydraDX
from pprint import pprint
import requests



import json, os


In [2]:

# TODO move to config file
arb_list = [
    {'tkns': ('HDX', 'USDT'), 'tkn_ids': (0, 10), 'order_book': ('HDX','USD')},
    {'tkns': ('HDX', 'USDT'), 'tkn_ids': (0, 23), 'order_book': ('HDX','USD')},
    {'tkns': ('HDX', 'USDT'), 'tkn_ids': (0, 2), 'order_book': ('HDX','USD')},
    {'tkns': ('HDX', 'USDT'), 'tkn_ids': (0, 7), 'order_book': ('HDX','USD')},
    {'tkns': ('HDX', 'USDT'), 'tkn_ids': (0, 18), 'order_book': ('HDX','USD')},
    {'tkns': ('HDX', 'USDT'), 'tkn_ids': (0, 21), 'order_book': ('HDX','USD')},
    {'tkns': ('HDX', 'USDT'), 'tkn_ids': (0, 22), 'order_book': ('HDX','USD')},
    {'tkns': ('DOT', 'USDT'), 'tkn_ids': (5, 10), 'order_book': ('DOT','USD')},
    {'tkns': ('DOT', 'USDT'), 'tkn_ids': (5, 23), 'order_book': ('DOT','USD')},
    {'tkns': ('DOT', 'USDT'), 'tkn_ids': (5, 2), 'order_book': ('DOT','USD')},
    {'tkns': ('DOT', 'USDT'), 'tkn_ids': (5, 7), 'order_book': ('DOT','USD')},
    {'tkns': ('DOT', 'USDT'), 'tkn_ids': (5, 18), 'order_book': ('DOT','USD')},
    {'tkns': ('DOT', 'USDT'), 'tkn_ids': (5, 21), 'order_book': ('DOT','USD')},
    {'tkns': ('DOT', 'USDT'), 'tkn_ids': (5, 22), 'order_book': ('DOT','USD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (4, 10), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (4, 23), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (4, 2), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (4, 7), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (4, 18), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (4, 21), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (4, 22), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (20, 10), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (20, 23), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (20, 2), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (20, 7), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (20, 18), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (20, 21), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('WETH', 'USDT'), 'tkn_ids': (20, 22), 'order_book': ('XETH','ZUSD')},
    {'tkns': ('DOT', 'WETH'), 'tkn_ids': (5, 4), 'order_book': ('DOT','ETH')},
    {'tkns': ('DOT', 'WETH'), 'tkn_ids': (5, 20), 'order_book': ('DOT','ETH')},
    # {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (3, 10), 'order_book': ('XXBT','ZUSD')},
    # {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (3, 23), 'order_book': ('XXBT','ZUSD')},
    # {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (3, 2), 'order_book': ('XXBT','ZUSD')},
    # {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (3, 7), 'order_book': ('XXBT','ZUSD')},
    # {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (3, 18), 'order_book': ('XXBT','ZUSD')},
    # {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (3, 21), 'order_book': ('XXBT','ZUSD')},
    # {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (3, 22), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (19, 10), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (19, 23), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (19, 2), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (19, 7), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (19, 18), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (19, 21), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('WBTC', 'USDT'), 'tkn_ids': (19, 22), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('IBTC', 'USDT'), 'tkn_ids': (11, 10), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('IBTC', 'USDT'), 'tkn_ids': (11, 23), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('IBTC', 'USDT'), 'tkn_ids': (11, 2), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('IBTC', 'USDT'), 'tkn_ids': (11, 7), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('IBTC', 'USDT'), 'tkn_ids': (11, 18), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('IBTC', 'USDT'), 'tkn_ids': (11, 21), 'order_book': ('XXBT','ZUSD')},
    {'tkns': ('IBTC', 'USDT'), 'tkn_ids': (11, 22), 'order_book': ('XXBT','ZUSD')},
    # {'tkns': ('DOT', 'WBTC'), 'tkn_ids': (5, 3), 'order_book': ('DOT','XBT')},
    {'tkns': ('DOT', 'WBTC'), 'tkn_ids': (5, 19), 'order_book': ('DOT','XBT')},
    {'tkns': ('DOT', 'IBTC'), 'tkn_ids': (5, 11), 'order_book': ('DOT','XBT')},
    # {'tkns': ('WETH', 'WBTC'), 'tkn_ids': (4, 3), 'order_book': ('XETH','XXBT')},
    {'tkns': ('WETH', 'WBTC'), 'tkn_ids': (4, 19), 'order_book': ('XETH','XXBT')},
    {'tkns': ('WETH', 'IBTC'), 'tkn_ids': (4, 11), 'order_book': ('XETH','XXBT')},
    # {'tkns': ('WETH', 'WBTC'), 'tkn_ids': (20, 3), 'order_book': ('XETH','XXBT')},
    {'tkns': ('WETH', 'WBTC'), 'tkn_ids': (20, 19), 'order_book': ('XETH','XXBT')},
    {'tkns': ('WETH', 'IBTC'), 'tkn_ids': (20, 11), 'order_book': ('XETH','XXBT')},
    {'tkns': ('ASTR', 'USDT'), 'tkn_ids': (9, 10), 'order_book': ('ASTR','USD')},
    {'tkns': ('ASTR', 'USDT'), 'tkn_ids': (9, 23), 'order_book': ('ASTR','USD')},
    {'tkns': ('ASTR', 'USDT'), 'tkn_ids': (9, 2), 'order_book': ('ASTR','USD')},
    {'tkns': ('ASTR', 'USDT'), 'tkn_ids': (9, 7), 'order_book': ('ASTR','USD')},
    {'tkns': ('ASTR', 'USDT'), 'tkn_ids': (9, 18), 'order_book': ('ASTR','USD')},
    {'tkns': ('ASTR', 'USDT'), 'tkn_ids': (9, 21), 'order_book': ('ASTR','USD')},
    {'tkns': ('ASTR', 'USDT'), 'tkn_ids': (9, 22), 'order_book': ('ASTR','USD')},
    {'tkns': ('CFG', 'USDT'), 'tkn_ids': (13, 10), 'order_book': ('CFG','USD')},
    {'tkns': ('CFG', 'USDT'), 'tkn_ids': (13, 23), 'order_book': ('CFG','USD')},
    {'tkns': ('CFG', 'USDT'), 'tkn_ids': (13, 2), 'order_book': ('CFG','USD')},
    {'tkns': ('CFG', 'USDT'), 'tkn_ids': (13, 7), 'order_book': ('CFG','USD')},
    {'tkns': ('CFG', 'USDT'), 'tkn_ids': (13, 18), 'order_book': ('CFG','USD')},
    {'tkns': ('CFG', 'USDT'), 'tkn_ids': (13, 21), 'order_book': ('CFG','USD')},
    {'tkns': ('CFG', 'USDT'), 'tkn_ids': (13, 22), 'order_book': ('CFG','USD')},
    {'tkns': ('BNC', 'USDT'), 'tkn_ids': (14, 10), 'order_book': ('BNC','USD')},
    {'tkns': ('BNC', 'USDT'), 'tkn_ids': (14, 23), 'order_book': ('BNC','USD')},
    {'tkns': ('BNC', 'USDT'), 'tkn_ids': (14, 2), 'order_book': ('BNC','USD')},
    {'tkns': ('BNC', 'USDT'), 'tkn_ids': (14, 7), 'order_book': ('BNC','USD')},
    {'tkns': ('BNC', 'USDT'), 'tkn_ids': (14, 18), 'order_book': ('BNC','USD')},
    {'tkns': ('BNC', 'USDT'), 'tkn_ids': (14, 21), 'order_book': ('BNC','USD')},
    {'tkns': ('BNC', 'USDT'), 'tkn_ids': (14, 22), 'order_book': ('BNC','USD')},
    {'tkns': ('GLMR', 'USDT'), 'tkn_ids': (16, 10), 'order_book': ('GLMR','USD')},
    {'tkns': ('GLMR', 'USDT'), 'tkn_ids': (16, 23), 'order_book': ('GLMR','USD')},
    {'tkns': ('GLMR', 'USDT'), 'tkn_ids': (16, 2), 'order_book': ('GLMR','USD')},
    {'tkns': ('GLMR', 'USDT'), 'tkn_ids': (16, 7), 'order_book': ('GLMR','USD')},
    {'tkns': ('GLMR', 'USDT'), 'tkn_ids': (16, 18), 'order_book': ('GLMR','USD')},
    {'tkns': ('GLMR', 'USDT'), 'tkn_ids': (16, 21), 'order_book': ('GLMR','USD')},
    {'tkns': ('GLMR', 'USDT'), 'tkn_ids': (16, 22), 'order_book': ('GLMR','USD')},
]

order_books = {}
for arb_cfg in arb_list:
    tkn_pair = arb_cfg['order_book']
    if tkn_pair not in order_books:
        resp = requests.get('https://api.kraken.com/0/public/Depth?pair=' + tkn_pair[0] + tkn_pair[1])
        y = resp.json()
        order_books[tkn_pair] = y['result'][tkn_pair[0] + tkn_pair[1]]
    


In [3]:
def get_unique_name(ls: list[str], name: str) -> str:
    if name not in ls:
        return name
    else:
        c = 1
        while name + str(c).zfill(3) in ls:
            c += 1
        return name + str(c).zfill(3)

In [4]:
RPC="wss://hydradx-rpc.dwellir.com"

chain = HydraDX(RPC) 
chain.connect()

# Omnipool metadata
# TODO: move to JSON file
metadata = [
    {'symbol': 'HDX', 'id': 0, 'decimals': 12},
    {'symbol': 'LRNA', 'id': 1, 'decimals': 12},
    {'symbol': 'DAI', 'id': 2, 'decimals': 18},
    {'symbol': 'WBTC', 'id': 3, 'decimals': 8},
    {'symbol': 'WETH', 'id': 4, 'decimals': 18},
    {'symbol': 'DOT', 'id': 5, 'decimals': 10},
    {'symbol': 'APE', 'id': 6, 'decimals': 18},
    {'symbol': 'USDC', 'id': 7, 'decimals': 6},
    {'symbol': 'PHA', 'id': 8, 'decimals': 12},
    {'symbol': 'ASTR', 'id': 9, 'decimals': 18},
    {'symbol': 'USDT', 'id': 10, 'decimals': 6},
    {'symbol': 'IBTC', 'id': 11, 'decimals': 8},
    {'symbol': 'ZTG', 'id': 12, 'decimals': 10},
    {'symbol': 'CFG', 'id': 13, 'decimals': 18},
    {'symbol': 'BNC', 'id': 14, 'decimals': 12},
    {'symbol': 'vDOT', 'id': 15, 'decimals': 10},
    {'symbol': 'GLMR', 'id': 16, 'decimals': 18},
    {'symbol': 'INTR', 'id': 17, 'decimals': 10},
    {'symbol': 'DAI', 'id': 18, 'decimals': 18},
    {'symbol': 'WBTC', 'id': 19, 'decimals': 8},
    {'symbol': 'WETH', 'id': 20, 'decimals': 18},
    {'symbol': 'USDC', 'id': 21, 'decimals': 6},
    {'symbol': 'USDC', 'id': 22, 'decimals': 6},
    {'symbol': 'USDT', 'id': 23, 'decimals': 6},
    ]

asset_list = []
fees = {}
tokens = {}
asset_map = {}

for i in range(len(metadata)):
    try:
        state = chain.api.omnipool.asset_state(metadata[i]['id'])
    except:
        continue
        
    fee = chain.api.fees.asset_fees(metadata[i]['id'])

    tkn = get_unique_name(asset_list, metadata[i]['symbol'])
    asset_list.append(tkn)
    asset_map[i] = tkn
    tokens[tkn] = {
        'liquidity': state.reserve / 10**metadata[i]['decimals'],
        'LRNA': state.hub_reserve / 10**12
    }
    fees[tkn] = {"asset_fee": fee.asset_fee / 100, "protocol_fee": fee.protocol_fee / 100}
        
chain.close()

pprint(tokens)


{'ASTR': {'LRNA': 9141.72948561221, 'liquidity': 4582987.0372328935},
 'BNC': {'LRNA': 7141.525088432134, 'liquidity': 705510.9117590818},
 'CFG': {'LRNA': 8762.7731450026, 'liquidity': 608267.9443819922},
 'DAI': {'LRNA': 13093.225661628376, 'liquidity': 333319.219548957},
 'DAI001': {'LRNA': 31719.080659534517, 'liquidity': 802549.3758030093},
 'DOT': {'LRNA': 70462.56757626562, 'liquidity': 425097.9504607403},
 'GLMR': {'LRNA': 5295.731249350026, 'liquidity': 671500.3249600258},
 'HDX': {'LRNA': 19279.014193867522, 'liquidity': 119345000.96126264},
 'IBTC': {'LRNA': 34887.54885040031, 'liquidity': 25.94363008},
 'USDT': {'LRNA': 84870.02270402151, 'liquidity': 2163268.968139},
 'WBTC': {'LRNA': 50.601798806111, 'liquidity': 0.04374934},
 'WBTC001': {'LRNA': 41825.972240324016, 'liquidity': 31.07194099},
 'WETH': {'LRNA': 21680.54627647201, 'liquidity': 307.38413849127414},
 'WETH001': {'LRNA': 71566.48323936212, 'liquidity': 1017.4471819398954},
 'ZTG': {'LRNA': 4669.488754083004, '

In [5]:

ob_objs = {}
for tkn_pair in order_books:
    ob = parse_kraken_orderbook(order_books[tkn_pair])
    ob_obj = OrderBook(
        bids=[[bid['price'], bid['amount']] for bid in ob['bids']],
        asks=[[ask['price'], ask['amount']] for ask in ob['asks']]
    )
    ob_objs[tkn_pair] = ob_obj
    
order_book_asset_list = []
for tkn_pair in order_books:
    for tkn in tkn_pair:
        if tkn not in order_book_asset_list:
            order_book_asset_list.append(tkn)

cex_fee = 0.0016
buffer = 0.0002

lrna_fee={asset: fees[asset]['protocol_fee'] for asset in asset_list}
asset_fee={asset: fees[asset]['asset_fee'] for asset in asset_list}

# lrna_fee = 0.0005
# asset_fee = 0.0025


cex = CentralizedMarket(
    order_book=ob_objs,
    asset_list=order_book_asset_list,
    trade_fee=cex_fee
)

op_state = OmnipoolState(
    tokens=tokens,
    lrna_fee = lrna_fee,
    asset_fee = asset_fee,
    preferred_stablecoin='USDT',
)

init_state = op_state.copy()


In [6]:
order_book_map = {}
for i in range(len(arb_list)):
    base_id, quote_id = arb_list[i]['tkn_ids']
    orderbook_tkn_pair = arb_list[i]['order_book']
    if base_id in asset_map and quote_id in asset_map:
        tkn_pair = (asset_map[base_id], asset_map[quote_id])
        order_book_map[tkn_pair] = orderbook_tkn_pair 
    
pprint(order_book_map)

{('ASTR', 'DAI'): ('ASTR', 'USD'),
 ('ASTR', 'DAI001'): ('ASTR', 'USD'),
 ('ASTR', 'USDT'): ('ASTR', 'USD'),
 ('BNC', 'DAI'): ('BNC', 'USD'),
 ('BNC', 'DAI001'): ('BNC', 'USD'),
 ('BNC', 'USDT'): ('BNC', 'USD'),
 ('CFG', 'DAI'): ('CFG', 'USD'),
 ('CFG', 'DAI001'): ('CFG', 'USD'),
 ('CFG', 'USDT'): ('CFG', 'USD'),
 ('DOT', 'DAI'): ('DOT', 'USD'),
 ('DOT', 'DAI001'): ('DOT', 'USD'),
 ('DOT', 'IBTC'): ('DOT', 'XBT'),
 ('DOT', 'USDT'): ('DOT', 'USD'),
 ('DOT', 'WBTC001'): ('DOT', 'XBT'),
 ('DOT', 'WETH'): ('DOT', 'ETH'),
 ('DOT', 'WETH001'): ('DOT', 'ETH'),
 ('GLMR', 'DAI'): ('GLMR', 'USD'),
 ('GLMR', 'DAI001'): ('GLMR', 'USD'),
 ('GLMR', 'USDT'): ('GLMR', 'USD'),
 ('HDX', 'DAI'): ('HDX', 'USD'),
 ('HDX', 'DAI001'): ('HDX', 'USD'),
 ('HDX', 'USDT'): ('HDX', 'USD'),
 ('IBTC', 'DAI'): ('XXBT', 'ZUSD'),
 ('IBTC', 'DAI001'): ('XXBT', 'ZUSD'),
 ('IBTC', 'USDT'): ('XXBT', 'ZUSD'),
 ('WBTC001', 'DAI'): ('XXBT', 'ZUSD'),
 ('WBTC001', 'DAI001'): ('XXBT', 'ZUSD'),
 ('WBTC001', 'USDT'): ('XXBT', 'ZUS

In [7]:
test_cex = cex.copy()
all_swaps = get_arb_swaps(op_state, test_cex, order_book_map, buffer=buffer)
pprint(all_swaps)
pprint(len(all_swaps))
# for tkn_pair in all_swaps:
#     if all_swaps[tkn_pair] != []:
#         print(tkn_pair)
#         print(all_swaps[tkn_pair])

[{'cex': {'amount': 2135.1243165626533,
          'buy_asset': 'USD',
          'price': 0.20197980000000001,
          'sell_asset': 'GLMR',
          'trade': 'sell'},
  'dex': {'amount': 2135.1243165626533,
          'buy_asset': 'GLMR',
          'max_sell': 428.9635449316502,
          'price': 0.202,
          'sell_asset': 'DAI001',
          'trade': 'buy'}},
 {'cex': {'amount': 0.8490788650512695,
          'buy_asset': 'ZUSD',
          'price': 1799.810001,
          'sell_asset': 'XETH',
          'trade': 'sell'},
  'dex': {'amount': 0.8490788650512695,
          'buy_asset': 'WETH001',
          'max_sell': 1950.5421714495897,
          'price': 1799.99,
          'sell_asset': 'DAI001',
          'trade': 'buy'}},
 {'cex': {'amount': 30.364,
          'buy_asset': 'BNC',
          'price': 0.2560256,
          'sell_asset': 'USD',
          'trade': 'buy'},
  'dex': {'amount': 30.364,
          'buy_asset': 'USDT',
          'min_buy': 7.809701334536076,
          'price

In [8]:
state = init_state.copy()
holdings = {asset: 1000000000 for asset in asset_list + order_book_asset_list}
init_agent = Agent(holdings=holdings, unique_id='bot')
agent = init_agent.copy()
new_cex = cex.copy()
execute_arb(state, new_cex, agent, all_swaps)

asset_map = {'WETH': 'ETH',
             'XETH': 'ETH',
             'XXBT': 'BTC',
             'WBTC': 'BTC',
             'ZUSD': 'USD',
             'USDT': 'USD',
             'USDC': 'USD',
             'DAI': 'USD',
             'USDT001': 'USD',
             'DAI001': 'USD',
             'WETH001': 'ETH',
             'WBTC001': 'BTC',
             'WETH001': 'ETH',
             'IBTC': 'BTC',
             'XBT': 'BTC',}

profit = calculate_profit(init_agent, agent, asset_map)

print(profit)
            

{'HDX': 0, 'USD': 6.401661396026611, 'BTC': 2.384185791015625e-07, 'ETH': 0.0, 'DOT': 0, 'ASTR': 0.0, 'ZTG': 0, 'CFG': 0.0, 'BNC': 0.0, 'vDOT': 0, 'GLMR': 0.0}
